Imports

In [1]:
import re
import bs4
import requests
import mysql.connector
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

Functions which are needed to scrap data from website

In [ ]:
# Use regular expression to extract the VIN from the string
def clean_vin(vin_text):
    vin_match = re.search(r'VIN[^\w]*(\w+)', vin_text, re.I)
    return vin_match.group(1) if vin_match else "VIN not available"

# Use regular expression to extract the number of accident and previous owners from the string
def accident_count(text):
    match = re.search(r'(\d+)\saccident', text, re.IGNORECASE)

    if match:
        return int(match.group(1))
    else:
        return 0

def owner_count(text):
    owner_pattern = r'(\d+)\sOwner'
    match = re.search(owner_pattern, text, re.IGNORECASE)

    if match:
        return int(match.group(1))
    else:
        return 0

Connect to MySQL server

In [ ]:
cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='vehicle_info')

cursor = cnx.cursor()

Gets the name of the car as input and find the number of pages of the cars

In [ ]:
car_name = input().split(' ')
car_url = '/'.join(car_name)
full_url = 'https://www.truecar.com/used-cars-for-sale/listings/{0}/'.format(car_url)

page = requests.get(full_url)
soup = bs4.BeautifulSoup(page.text, 'html.parser')
number_of_pages = int(soup.find_all('a', attrs={'data-qa': 'Pagination-link'})[-1].text)

Iterate over all pages and extracts the information of each car using its unique VIN(Vehicle Identification Number), and adds the cars attributes to the car_info table. Omitting cars with not available prices at the end.

In [ ]:
cursor = cnx.cursor()

car_model_number = dict()
n = 1
cars = dict()

for page_num in range(1, number_of_pages + 1):
    full_url_each_page = f'{full_url}?page={page_num}'
    page = requests.get(full_url_each_page)
    soup = bs4.BeautifulSoup(page.text, 'html.parser')

    car_tags = soup.find_all('div', attrs = {'class' : 'vehicle-card-vin-carousel mt-1 text-xs'})

    for vin_element in car_tags:
        each_car = list()
        
        if vin_element:
            car_vin = vin_element.text
        else:
            car_vin = "VIN not available"

        car_model_element = vin_element.find_previous('div', attrs={'data-test': 'vehicleCardTrim'})
        car_model = car_model_element.text.strip() if car_model_element else 'N/A'
        
        # Assign a number to each model of car
        if not car_model in car_model_number:
            car_model_number[car_model] = n
            n += 1
        each_car_model_num = car_model_number[car_model]

        # Extract car year_model
        car_year_model_element = vin_element.find_previous('span', class_='vehicle-card-year text-xs')
        car_year_model = int(car_year_model_element.text.strip()) if car_year_model_element and car_year_model_element.text.strip().isdigit() else 'N/A'

        # Extract car mileage
        car_mileage_element = vin_element.find_previous('div', attrs={'data-test': 'vehicleMileage'})
        car_mileage = int(''.join(filter(str.isdigit, car_mileage_element.text.strip()))) if car_mileage_element else 'N/A'

        # Extract car number of accidents and owners
        car_condition_text = vin_element.find_previous('div', attrs={'data-test': "vehicleCardCondition"})
        car_accidents = accident_count(car_condition_text.text)
        car_owners = owner_count(car_condition_text.text)

        # Extract car price
        price_element = vin_element.find_previous('div', attrs = {"vehicle-card-bottom-pricing-secondary pl-3 lg:pl-2 vehicle-card-bottom-max-50"})
        each_car_price_element = price_element.find('span', attrs={'data-test': 'vehicleListingPriceAmount'})
        price = int(''.join(filter(str.isdigit, each_car_price_element.text.strip()))) if each_car_price_element else 'N/A'

        # Create a dictionary of all cars and their attributes
        each_car.append(car_model)
        each_car.append(each_car_model_num)
        each_car.append(car_year_model)
        each_car.append(car_mileage)
        each_car.append(car_accidents)
        each_car.append(car_owners)
        each_car.append(price)

        cars[vin_element] = each_car

        # Add all info to a table in sql
        query = 'INSERT INTO car_info (vin, model, model_number, year_model, mileage, accidents, owners, price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
        values = (car_vin, car_model, each_car_model_num, car_year_model, car_mileage, car_accidents, car_owners, price)
        cursor.execute(query, values)
        cnx.commit()

        query_2 = 'DELETE FROM car_info WHERE price LIKE \'N/A\''
        cursor.execute(query_2)
        cnx.commit()

Some MySQL queries for creating table and edit it

In [ ]:
    # CREATE DATABASE vehicle_info;
    # SHOW DATABASES;
    # USE vehicle_info
    # CREATE TABLE car_info (vin varchar(30), model varchar(60), model_number int, year_model int, mileage varchar(10), accidents int, owners int, price varchar(10));
    # DESC car_info;
    # SELECT * FROM car_info;
    # DELETE FROM car_info;
    # DROP TABLE car_info;

Creating a CSV file from SQL table

In [ ]:
#turn sql table into csv file
cursor.execute('SELECT * FROM car_info')
rows = cursor.fetchall()

header = [row[0] for row in cursor.description]

with open("D:\Learning\Python\Jadi-Advance\Final project\car_info.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for row in rows:
        writer.writerow(row)

cnx.close()

Create a dataframe from CSV file and then filtering cars with less than 4 times repetitions. This filtering is applied to let stratify distribute the cars between train and test datasets in the right proportion.

In [ ]:
# Creating Dataframe
df = pd.read_csv("D:\Learning\Python\Jadi-Advance\Final project\car_info.csv")
# print(df['model_number'].value_counts())

df = df[df['model_number']]
target = df['price'].to_numpy()
final_df = df.drop(['vin', 'price'], axis= 1)

# Filtering car models that repeated less than 4 times to stratify works properly
final_df = final_df.groupby('model_number').filter(lambda x: len(x) > 4)
# print(final_df['model_number'].value_counts())

Create train and test datasets and Scale the numerical attributes and encode the categorical ones and then concatenate these two types of attributes.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df, target, test_size=0.3, stratify= final_df['model_number'].values)
X_train_categorical = X_train[['model']].to_numpy().reshape(-1, 1)
X_train_numerical = X_train[['year_model', 'mileage']].to_numpy()
X_test_categorical = X_test[['model']].to_numpy().reshape(-1, 1)
X_test_numerical = X_test[['year_model', 'mileage']].to_numpy()

# Scaling
scaler = StandardScaler()
X_train_numerical = scaler.fit_transform(X_train_numerical)
X_test_numerical = scaler.transform(X_test_numerical)
data_scaler = pickle.dumps(scaler)

enc = OneHotEncoder(handle_unknown='ignore')
X_train_categorical = enc.fit_transform(X_train_categorical).toarray()
X_test_categorical = enc.transform(X_test_categorical).toarray()
data_encoder = pickle.dumps(enc)

X_train = np.concatenate((X_train_categorical, X_train_numerical), axis=1)
X_test = np.concatenate((X_test_categorical, X_test_numerical), axis=1)

Train a linear regression model on the train dataset and then testing it

In [ ]:
# Train a ML model Regression and testing it
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
ml_model = pickle.dumps(reg)

print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

Function for predicting new cars price.

In [ ]:
# Defining a function to predict the price of new cars
def predict_price(car_info):
    car_categorical = np.array([car_info[0]]).reshape(-1, 1)
    car_numerical = np.array([car_info[1:]])
    enc = pickle.loads(data_encoder)
    scaler = pickle.loads(data_scaler)
    price_predictor = pickle.loads(ml_model)

    X_numerical = scaler.transform(car_numerical)
    X_categorical = enc.transform(car_categorical).toarray()

    X = np.concatenate((X_categorical, X_numerical), axis=1)
    price_pred = price_predictor.predict(X)
    return price_pred

In [ ]:
new_car = input('please enter the model of car its manufacture year and mileage and separate them by \',\': ').split(',')
new_car_info = [new_car[0]] + [int(x) for x in new_car[1:]]
new_car_price = predict_price(new_car_info)
print(new_car_price)